# COVID-19 Classification using 4-layer Convolutional Neural Networks.

In [2]:
! pip install opencv-python

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import numpy as np
import shutil
from sklearn import preprocessing
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import f1_score
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from glob import glob

Using TensorFlow backend.


In [4]:
base_dir = './data/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

In [5]:
import tensorflow as tf
 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(3, activation='softmax')  
])


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [7]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150),
                                                         shuffle=False)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [9]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=10,
                              epochs=20,
                              validation_steps = 3,
                              verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 3 steps
Epoch 1/20
10/10 - 10s - loss: 1.7097 - accuracy: 0.4921 - val_loss: 0.8808 - val_accuracy: 0.7167
Epoch 2/20
10/10 - 9s - loss: 0.6634 - accuracy: 0.8691 - val_loss: 0.4274 - val_accuracy: 0.8667
Epoch 3/20
10/10 - 9s - loss: 0.3093 - accuracy: 0.9005 - val_loss: 0.1908 - val_accuracy: 0.9167
Epoch 4/20
10/10 - 10s - loss: 0.2373 - accuracy: 0.8953 - val_loss: 0.2648 - val_accuracy: 0.8833
Epoch 5/20
10/10 - 9s - loss: 0.1180 - accuracy: 0.9529 - val_loss: 0.2770 - val_accuracy: 0.8667
Epoch 6/20
10/10 - 9s - loss: 0.1137 - accuracy: 0.9476 - val_loss: 0.4262 - val_accuracy: 0.7833
Epoch 7/20
10/10 - 10s - loss: 0.0711 - accuracy: 0.9581 - val_loss: 0.2823 - val_accuracy: 0.9000
Epoch 8/20
10/10 - 10s - loss: 0.0625 - accuracy: 0.9791 - val_loss: 0.1755 - val_accuracy: 0.9333
Epoch 9/20
10/10 - 11s - loss: 0.0443 - accuracy: 0.9900 - val_loss: 0.1840 - val_accuracy: 0.9167
Epoch 10/20
10/10 - 9s

In [11]:
from sklearn.metrics import classification_report
from sklearn import preprocessing

y_pred = model.predict(validation_generator, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool))

4/4 [==============================] - 1s 318ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      0.70      0.82        20
           2       0.77      1.00      0.87        20

    accuracy                           0.91        66
   macro avg       0.92      0.90      0.90        66
weighted avg       0.93      0.91      0.91        66

